In [1]:
import numpy as np
import pandas as pd
from plotnine import *
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'NanumGothicCoding'
plt.rcParams['axes.unicode_minus'] = False

from glob import glob
import re
import math

출처: 해양환경정보> 해양환경 관측&조사> 해양환경측정망 정보 
* 원본자료 
https://www.meis.go.kr/mei/observe/port.do#

* 15~18년의 수질정보를 전처리
* 15~16에서는 수질등급을 직접 계산하여 넣어줘야함


In [3]:
# 수질평가등급을 계산하기위한 함수
# https://www.meis.go.kr/mei/wqi/introduce.do

# 저층산소포화도를 계산하기 위한 용존산소 포화농도(Ds)를 계산하는 함수 
# T는 수온, S는 염도를 나타낸다.

def aaa(t, S) : 
    T = 273.15 + t
    
    C = -173.4292 + 249.6339*(10**2 * T**-1) + 143.3483 * math.log(T * 10**-2) -21.8492*(T * 10**-2) -S*(0.033096 - 0.014259* T *10**-2 + 0.0017 * T**2 * 10**-4 )
    c = math.e**C
    result = c*32/22.4
    return result

# 수질을 계산시 각 항목을 1~5까지 분류하여 계산
# 항목을 5단계로 분류하는 함수
# 해협마다 기준치가 다르기 때문에 조건문을 통해서 구분

# 클로로필(Chl-a) 분류
def chl_convert(a1):
    if a.loc[a1,'생태구역'] =='동해':
        b = 2.1
        if chl[a1] <= b :
            return 1
        elif (chl[a1] > b)&(chl[a1] <= b + b*0.1):
            return 2
        elif (chl[a1] > b + b*0.1)&(chl[a1] <= b + b*0.25) :
            return 3
        elif (chl[a1] > b + b*0.25)&(chl[a1] <= b + b*0.50) :
            return 4
        else :
            return 5
        
    elif a.loc[a1,'생태구역'] =='대한해협':
        b = 6.3
        if chl[a1] <= b :
            return 1
        elif (chl[a1] > b)&(chl[a1] <= b + b*0.1):
            return 2
        elif (chl[a1] > b + b*0.1)&(chl[a1] <= b + b*0.25) :
            return 3
        elif (chl[a1] > b + b*0.25)&(chl[a1] <= b + b*0.50) :
            return 4
        else :
            return 5
    elif a.loc[a1,'생태구역'] =='서남해역':
        b = 3.7
        if chl[a1] <= b :
            return 1
        elif (chl[a1] > b)&(chl[a1] <= b + b*0.1):
            return 2
        elif (chl[a1] > b + b*0.1)&(chl[a1] <= b + b*0.25) :
            return 3
        elif (chl[a1] > b + b*0.25)&(chl[a1] <= b + b*0.50) :
            return 4
        else :
            return 5
    elif a.loc[a1,'생태구역'] =='제주':
        b = 1.6
        if chl[a1] <= b :
            return 1
        elif (chl[a1] > b)&(chl[a1] <= b + b*0.1):
            return 2
        elif (chl[a1] > b + b*0.1)&(chl[a1] <= b + b*0.25) :
            return 3
        elif (chl[a1] > b + b*0.25)&(chl[a1] <= b + b*0.50) :
            return 4
        else :
            return 5
    elif a.loc[a1,'생태구역'] =='서해중부':
        b = 2.2
        if chl[a1] <= b :
            return 1
        elif (chl[a1] > b)&(chl[a1] <= b + b*0.1):
            return 2
        elif (chl[a1] > b + b*0.1)&(chl[a1] <= b + b*0.25) :
            return 3
        elif (chl[a1] > b + b*0.25)&(chl[a1] <= b + b*0.50) :
            return 4
        else :
            return 5

# 산소포화도(DO/Ds) 분류 
# 산소포화도는 모든 지역이 같은 기준을 가짐 
def do_convert(a1):
        b = 0.9
        if do[a1] >= b :
            return 1
        elif (do[a1] < b)&(do[a1] >= b - b*0.1):
            return 2
        elif (do[a1] < b - b*0.1)&(do[a1] >= b - b*0.25) :
            return 3
        elif (do[a1] < b - b*0.25)&(do[a1] >= b - b*0.50) :
            return 4
        else :
            return 5       

# 용존 무기인(DIP) 분류
def dip_convert(a1):
    if a.loc[a1,'생태구역'] =='동해':
        b = 20
        if dip[a1] <= b :
            return 1
        elif (dip[a1] > b)&(dip[a1] <= b + b*0.1):
            return 2
        elif (dip[a1] > b + b*0.1)&(dip[a1] <= b + b*0.25) :
            return 3
        elif (dip[a1] > b + b*0.25)&(dip[a1] <= b + b*0.50) :
            return 4
        else :
            return 5
        
    elif a.loc[a1,'생태구역'] =='대한해협':
        b = 35
        if dip[a1] <= b :
            return 1
        elif (dip[a1] > b)&(dip[a1] <= b + b*0.1):
            return 2
        elif (dip[a1] > b + b*0.1)&(dip[a1] <= b + b*0.25) :
            return 3
        elif (dip[a1] > b + b*0.25)&(dip[a1] <= b + b*0.50) :
            return 4
        else :
            return 5
    elif a.loc[a1,'생태구역'] =='서남해역':
        b = 25
        if dip[a1] <= b :
            return 1
        elif (dip[a1] > b)&(dip[a1] <= b + b*0.1):
            return 2
        elif (dip[a1] > b + b*0.1)&(dip[a1] <= b + b*0.25) :
            return 3
        elif (dip[a1] > b + b*0.25)&(dip[a1] <= b + b*0.50) :
            return 4
        else :
            return 5
    elif a.loc[a1,'생태구역'] =='제주':
        b = 15
        if dip[a1] <= b :
            return 1
        elif (dip[a1] > b)&(dip[a1] <= b + b*0.1):
            return 2
        elif (dip[a1] > b + b*0.1)&(dip[a1] <= b + b*0.25) :
            return 3
        elif (dip[a1] > b + b*0.25)&(dip[a1] <= b + b*0.50) :
            return 4
        else :
            return 5
    elif a.loc[a1,'생태구역'] =='서해중부':
        b = 30
        if dip[a1] <= b :
            return 1
        elif (dip[a1] > b)&(dip[a1] <= b + b*0.1):
            return 2
        elif (dip[a1] > b + b*0.1)&(dip[a1] <= b + b*0.25) :
            return 3
        elif (dip[a1] > b + b*0.25)&(dip[a1] <= b + b*0.50) :
            return 4
        else :
            return 5

# 용존 무기질소(DIN) 분류
def din_convert(a1):
    if a.loc[a1,'생태구역'] =='동해':
        b = 140
        if din[a1] <= b :
            return 1
        elif (din[a1] > b)&(din[a1] <= b + b*0.1):
            return 2
        elif (din[a1] > b + b*0.1)&(din[a1] <= b + b*0.25) :
            return 3
        elif (din[a1] > b + b*0.25)&(din[a1] <= b + b*0.50) :
            return 4
        else :
            return 5
        
    elif a.loc[a1,'생태구역'] =='대한해협':
        b = 220
        if din[a1] <= b :
            return 1
        elif (din[a1] > b)&(din[a1] <= b + b*0.1):
            return 2
        elif (din[a1] > b + b*0.1)&(din[a1] <= b + b*0.25) :
            return 3
        elif (din[a1] > b + b*0.25)&(din[a1] <= b + b*0.50) :
            return 4
        else :
            return 5
    elif a.loc[a1,'생태구역'] =='서남해역':
        b = 230
        if din[a1] <= b :
            return 1
        elif (din[a1] > b)&(din[a1] <= b + b*0.1):
            return 2
        elif (din[a1] > b + b*0.1)&(din[a1] <= b + b*0.25) :
            return 3
        elif (din[a1] > b + b*0.25)&(din[a1] <= b + b*0.50) :
            return 4
        else :
            return 5
    elif a.loc[a1,'생태구역'] =='제주':
        b = 165
        if din[a1] <= b :
            return 1
        elif (din[a1] > b)&(din[a1] <= b + b*0.1):
            return 2
        elif (din[a1] > b + b*0.1)&(din[a1] <= b + b*0.25) :
            return 3
        elif (din[a1] > b + b*0.25)&(din[a1] <= b + b*0.50) :
            return 4
        else :
            return 5
    elif a.loc[a1,'생태구역'] =='서해중부':
        b = 425
        if chl[a1] <= b :
            return 1
        elif (din[a1] > b)&(din[a1] <= b + b*0.1):
            return 2
        elif (din[a1] > b + b*0.1)&(din[a1] <= b + b*0.25) :
            return 3
        elif (din[a1] > b + b*0.25)&(din[a1] <= b + b*0.50) :
            return 4
        else :
            return 5

# 투명도(W) 분류
def w_convert(a1):
    if a.loc[a1,'생태구역'] =='동해':
        b = 8.5
        if w[a1] >= b :
            return 1
        elif (w[a1] < b)&(w[a1] >= b - b*0.1):
            return 2
        elif (w[a1] < b- b*0.1)&(w[a1] >= b - b*0.25) :
            return 3
        elif (w[a1] < b - b*0.25)&(w[a1] >= b - b*0.50) :
            return 4
        else :
            return 5
        
    elif a.loc[a1,'생태구역'] =='대한해협':
        b = 2.5
        if w[a1] >= b :
            return 1
        elif (w[a1] < b)&(w[a1] >= b - b*0.1):
            return 2
        elif (w[a1] < b- b*0.1)&(w[a1] >= b - b*0.25) :
            return 3
        elif (w[a1] < b - b*0.25)&(w[a1] >= b - b*0.50) :
            return 4
        else :
            return 5
    elif a.loc[a1,'생태구역'] =='서남해역':
        b = 0.5
        if w[a1] >= b :
            return 1
        elif (w[a1] < b)&(w[a1] >= b - b*0.1):
            return 2
        elif (w[a1] < b- b*0.1)&(w[a1] >= b - b*0.25) :
            return 3
        elif (w[a1] < b - b*0.25)&(w[a1] >= b - b*0.50) :
            return 4
        else :
            return 5
    elif a.loc[a1,'생태구역'] =='제주':
        b = 8.0
        if w[a1] >= b :
            return 1
        elif (w[a1] < b)&(w[a1] >= b - b*0.1):
            return 2
        elif (w[a1] < b- b*0.1)&(w[a1] >= b - b*0.25) :
            return 3
        elif (w[a1] < b - b*0.25)&(w[a1] >= b - b*0.50) :
            return 4
        else :
            return 5
    elif a.loc[a1,'생태구역'] =='서해중부':
        b = 1.0
        if w[a1] >= b :
            return 1
        elif (w[a1] < b)&(w[a1] >= b - b*0.1):
            return 2
        elif (w[a1] < b- b*0.1)&(w[a1] >= b - b*0.25) :
            return 3
        elif (w[a1] < b - b*0.25)&(w[a1] >= b - b*0.50) :
            return 4
        else :
            return 5


# 수질평가지수 계산식
# a = DO, b= Chl-a, c= W, d=DIN, e=DIP
def cul(a,b,c,d,e):
    sum = 10*a+6*(b+c)/2+4*(d+e)/2
    return sum

# 수질평가지수를 5단계로 분류하는 함수
def classfy(i):
    if i <=23:
        return 1
    elif (i >23)&(i<=33):
        return 2
    elif (i >33)&(i<=46):
        return 3
    elif (i >46)&(i<=59):
        return 4
    elif (i >59):
        return 5


    



In [7]:
# 사전 excel에서 사용하지 않는 컬럼제거, 수질등급을 넣을 컬럼 생성
# 지역명이 멀티컬럼으로 구성되어 NaN값이 발생
# st.No 컬럼을 기준으로 NaN값을 채워줌
file_list = []
for i in range(2015,2017):
    a = '../../data/team/해양환경측정망_'+str(i)+'.xlsx'
    file_list.append(a)

key = ['2월','5월','8월','11월']
con_data = pd.DataFrame()
for i in file_list :
    # 엑셀시트가 4개 존재함으로 sheet_name을 None으로 두어 전부 호출
    # 이 경우 딕셔너리 타입으로 호출됨
    df_all = pd.read_excel(i, skiprows = 3, header=None, sheet_name=None)
    
    # 저장한 키값을 통해서 시트 호출
    for j in key:
        # 월별시트 호출
        data = df_all[j]
        
        # st.No에서 h제거
        h= data[data[4].str.contains('H', na=False)]
        h[4] = h[4].str.get(i=1)
        h1 = h.index
        for h_1 in range(len(h1)):
            data.loc[h1[h_1],4] = h.loc[h1[h_1],4]
        
        # 해역컬럼에서 불필요한 단어제거
        man = data[data[2].str.contains('\n', na=False)]
        man[2] = man[2].str[-3:-1]
        h2 = man.index
        for h_2 in h2:
            data.loc[h_2,2] = man.loc[h_2,2]
            
        
        # 1로 시작하는 st.No의 인덱스 추출
        index = data[data[4] == 1].index
        
        # index, st.No를 통해 빈칸채우기
        for z in range(len(index)):  
            insert_l = data.loc[index[z],:3]
            
            # 마지막인덱스 이외의 인덱스
            # 데이터를 넣을 시 다음 인덱스까지 범위를 지정하기 때문에
            # 마지막 인덱스를 제외
            if index[z] < index[len(index)-1]:
                
                # 항만이 이중으로 열병합 되어있는 경우 0:3까지 nan처리됨
                # nan값은 nan == nan 일시 Flase가 나옴
                # 0:3까지의 값은 이전 인덱스의 값과 동일하기 때문에 이전 인덱스의 값 호출
                if data.loc[index[z],3] != data.loc[index[z],3]:
                    for k in range(index[z],index[z+1]):
                        insert_l_1 = data.loc[index[z]-1,:3]
                        data.loc[k,:3] = insert_l_1
                else :
                    # 월이 이중으로 열병합 되어있는 경우
                    if data.loc[index[z],0] != data.loc[index[z],0]:
                        insert_l_1 = data.loc[index[z]-1,:2]
                        for a_1 in range(index[z],len(data)):
                            data.loc[a_1,:2] = insert_l_1
                    else:
                        
                        for a in range(index[z],index[z+1]):
                            data.loc[a,:3] = insert_l
                        
            # 마지막 인덱스
            else : 
                if data.loc[index[z],3] != data.loc[index[z],3]:
                    for p in range(index[z],len(data)):
                        insert_l_1 = data.loc[index[z]-1,:3]
                        data.loc[p,:3] = insert_l_1
                else:
                    
                    if data.loc[index[z],0] != data.loc[index[z],0]:
                        insert_l_1 = data.loc[index[z]-1,:2]
                        for e_1 in range(index[z],len(data)):
                            data.loc[e_1,:2] = insert_l_1
                    else: 
                        
                        for e in range(index[z],len(data)):
                            data.loc[e,:3] = insert_l
                    
                    
        con_data = pd.concat([con_data, data])
        # 빈칸채우기 끝
    

In [8]:
con_data.to_csv('../../data/team/1516.csv', index=False, encoding = 'utf-8')
con_data.to_csv('../../data/team/1516_r.csv', index=False, encoding = 'ANSI')

In [42]:
df = pd.read_csv('../../data/team/1516.csv', encoding='utf-8')

# 멀티컬럼이기 때문에 컬럼명을 전처리해야함
# 엑셀파일을 통해 컬럼의 이름을 전처리하고 이를 호출

index_df = pd.read_excel('../../data/team/해양환경측정망_2019.xlsx')
index_list = index_df.loc[0,:]

for i in range(37):
    df.rename(columns = {str(i): index_list[i]}, inplace = True)

In [43]:
b = df
# 생태구역 입력형식을 통일
for i in range(len(b)):
    if b.loc[i,'생태구역'] =='동 해':
        b.loc[i,'생태구역'] = '동해'
    if b.loc[i,'생태구역'] =='서 해':
        b.loc[i,'생태구역'] = '서해'
    if b.loc[i,'생태구역'] =='남 해':
        b.loc[i,'생태구역'] = '남해'
        

In [52]:
b['해수수질기준']=0
b= b.dropna()

In [53]:
b.to_csv('../../data/team/1516_1.csv', index=False, encoding = 'utf-8')
b.to_csv('../../data/team/1516_1_R.csv', index=False, encoding = 'ANSI')

In [54]:
a = pd.read_csv('../../data/team/1516_1.csv', encoding='utf-8', engine='python')
c = pd.read_csv('../../data/team/해양환경측정망_합본_1.csv', encoding='utf-8', engine='python')

In [55]:
list_a = c['연안명칭'].unique()[:65]

In [56]:
# 서해, 남해를 서남해역 등으로 수정
for i in range(len(a)):
    for j in list_a:
        if a.loc[i,'연안명칭'] == j:
            a.loc[i,'생태구역'] = c[c['연안명칭'] == j]['생태구역'].iloc[0]

In [9]:
# 수질분류
# 농도계산
a['Ds'] = ''
for i in range(len(a)):
    a.loc[i,'Ds'] = aaa(a.loc[i,'저층수온'],a.loc[i,'저층염분'])
do = a['저층DO']/a['Ds']

din = a['표층DIN']
dip = a['표층DIP']
chl = a['표층Chl-a']
w = a['표층투명도']

chl_a = []
din_a = []
dip_a = []
w_a = []
do_a = []

# 등급별 분류
for i in range(len(a)):
    chl_a.append(chl_convert(i))
    din_a.append(din_convert(i))
    dip_a.append(dip_convert(i))
    w_a.append(w_convert(i))
    do_a.append(do_convert(i))
    
su = []
# 해수수질등급 계산
for i in range(len(a)):
    su.append(cul(do_a[i],chl_a[i],w_a[i],din_a[i],dip_a[i]))

# 데이터프레임에 적용
for i in range(len(a)):
    a.loc[i,'해수수질기준'] = classfy(su[i])

TypeError: 'int' object does not support item assignment

In [ ]:
loc_list = a['연안명칭'].unique()
list_name = ['연안','만','하구','만','.자란만','동안','남안','외안','호']
a['구군'] = a['연안명칭']
for j in list_name:
    name_length = a['연안명칭'].str.find(j)

    for i in name_length.index :
        if name_length[i] != -1:
            a.loc[i,'구군'] = a.loc[i, '연안명칭'][0:name_length[i]]
        

In [314]:
a.to_csv('../../data/team/해양환경측정망_15,16.csv',index=False, encoding='utf-8')
a.to_csv('../../data/team/해양환경측정망_15,16_R.csv',index=False, encoding='ANSI')